In [1]:
import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn
import xarray as xr
from shapely.geometry import shape, Polygon
from affine import Affine
from rasterio import features
import geopandas as gpd
import geoviews as gv
import cartopy.crs as ccrs
from bokeh.models import HoverTool
import holoviews as hv
from holoviews import streams

In [2]:
pn.extension(design="material", sizing_mode="stretch_width")

In [3]:
def get_gdf_from_segment_grid(segments):
    x = segments.lon.values
    y = segments.lat.values
    labels = segments.values
    dx = float(np.mean(np.diff(x))) 
    dy = float(np.mean(np.diff(y))) 
    x0 = x.min() - dx / 2.0
    y0 = y.min() - dy / 2.0  
    transform = Affine.translation(x0, y0) * Affine.scale(dx, dy)
    geoms = []
    vals = []
    for geom, value in features.shapes(labels, transform=transform):
        if value == 0:
            continue
        geoms.append(shape(geom))
        vals.append(value)
    gdf_segments = gpd.GeoDataFrame(
        {"label": vals},
        geometry=geoms,
        crs="EPSG:32662"   # or "EPSG:3995" or your custom proj string
    )
    gdf_segments= gdf_segments.set_index('label') 
    gdf_segments=gdf_segments.reset_index().dissolve(by='label')
    return gdf_segments

In [4]:
K39_regions = {
1: 'Turkey',
2: 'Jan-Mayen islands',# Uninhabited: not used
3: 'Aegean + SW Greece',
4: 'North Norway + North Finland',
5: 'Central Norway',
6: 'Finnmark',
7: 'West Ukraine',
8: 'East Continental Europe',
9: 'Central Continental Europe',
10: 'North East Europe',
11: 'North Adriatic',
12: 'Central Alpine',
13: 'Northern North Sea',
14: 'West Norway',
15: 'West Libya',
16: 'Egypt + East Libya',
17: 'North Sweden + North Finland',
18: 'Central Sweden',
19: 'East Baltic',
20: 'West Baltic',
21: 'Corsica + Sardinia',
22: 'Tell Atlas',
23: 'Sicily + Calabria',
24: 'South Tunisia + East Algeria',
25: 'Central Algeria',
26: 'West Algeria + Balearics',
27: 'South Morocco + South West Algeria',
28: 'South East Spain + North Morocco',
29: 'North East Spain',
30: 'Southern France + Coastal North Spain',
31: 'NW Iberia',
32: 'SW Iberia + Tangier-Tétouan',
33: 'West France + South England',
34: 'Low Countries + Western Germany',
35: 'Central England + Wales + Ireland',
36: 'Southern North Sea',
37: 'Balkan + SE Europe',
38: 'South Adriatic',
39: 'North Aegean'
}


In [5]:
@pn.cache
def get_data():
    ds1= xr.open_dataset("/Data/gfi/users/jodor4442/cmip_precursor_metrics/LENS_precomputed_terms_combined.nc")
    ds1 = ds1.assign_coords(region_id = [K39_regions[k] for k in ds1.region_id.values])
    ds1 = xr.concat([ds_sel_barplot,ds_sel_barplot.sum('source').assign_coords(source='all')], dim='source') 
    regions = xr.open_dataset('/Data/gfi/share/ModData/CMIP_EU_Precip_Precursors/aux/ERA5_rainfall_regions.nc').tp
    gdf_regions = get_gdf_from_segment_grid(regions)#.to_crs({'init': 'epsg:4326'},inplace=True) 
    gdf_regions.index = [K39_regions[k] for k in gdf_regions.index]
    

    return ds1, gdf_regions

ds1, gdf_regions = get_data()
# data.tail()


In [6]:
def get_plot(model='CESM2', season='DJF', term='bias', source='dyn', vmax=50):
    """Plots the rolling average and the outliers"""
    ds_sel = (
        ds1
        .sel(model=model, season=season, term=term, source=source)
        .sum("synoptic_bin") / 0.05 * 100
    )

    gdf = gdf_regions.copy()
    gdf[term] = ds_sel.individual_term.to_series()
    
    # hover = HoverTool(tooltips=tooltips)
    hover = HoverTool(tooltips = [('@index', '@term')])
    poly_plot = gdf.hvplot.polygons( crs=ccrs.PlateCarree(), color=term)\
        .opts(height=800, 
              width=800, 
              # tools=['hover'],
              tools=['tap', 'hover'],  # Ensure tap is here
              # selection_mode='single',   # Or 'multiple'
              cmap = 'RdBu',
              color=term, 
              colorbar=True, 
              clim=(-vmax, vmax),
              projection=ccrs.PlateCarree(),
              axiswise=False, 
              shared_axes=True
             )#+gv.feature.ocean
    selection = streams.Selection1D(source=poly_plot)
    @pn.depends(selection.param.index)
    def update_secondary_plot(index):
        if not index:
            return "### Please select a region"
        # print(index)
        regions = gdf.iloc[index].index
        region_names='  \n'+'  \n'.join(regions)
        # data=gdf.loc[index].index
        ds_sel_barplot = (
                ds1
                .sel(model=model, season=season, term=term,  region_id=regions)
                .sum("synoptic_bin") / 0.05 * 100
            ).individual_term
        series =  ds_sel_barplot.to_series()#
        series.index = series.index.reorder_levels([1,0])
        bars = series.hvplot.bar(title=term)
        return bars.opts(height=800, 
                         width=200)*hv.HLine(0).opts(color='k')#f"### You clicked on:  \n{region_names}"

    
    layout = pn.Row(poly_plot, update_secondary_plot)
    return layout

In [7]:
# get_plot()

In [8]:
model_widget = pn.widgets.RadioButtonGroup(
    description="Model selection",
    name="Model",
    options=list(ds1.model.values),
)
season_widget = pn.widgets.RadioButtonGroup(
    description="Season selection",
    name="Season",
    options=list(ds1.season.values),
)
term_widget = pn.widgets.RadioButtonGroup(
    description="Term selection",
    name="Term",
    options=['bias','trend'],
)
source_widget = pn.widgets.RadioButtonGroup(
    description="Source selection",
    name="Source",
    options=list(ds1.source.values),
)
vmax_widget = pn.widgets.IntSlider(name="Colormap max value", value=50, start=1, end=150)

In [9]:
bound_plot = pn.bind(
    get_plot, model=model_widget, season=season_widget, term=term_widget, source=source_widget, vmax=vmax_widget
)

In [10]:
app = pn.template.MaterialTemplate(
    site="Panel",
    title="CMIP European rainfall decomposition",
    sidebar=[term_widget, source_widget, season_widget,model_widget, vmax_widget],
    main=[bound_plot],
)

In [11]:
# bound_plot

In [12]:
app.servable()

MaterialTemplate
    [js_area] HTML(None, design=<class 'panel.theme.materi..., height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] MaterialTemplateActions()
    [browser_info] BrowserInfo(dark_mode=False, device_pixel_ratio=2)
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-139847738130016] ParamFunction(function, _pane=Row, defer_load=False, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [nav-139847733900016] RadioButtonGroup(description='Term selection', design=<class 'panel.theme.materi..., name='Term', options=['bias', 'trend'], sizing_mode='stretch_width', value='bias')
    [nav-139847738960560] RadioButtonGroup(description='Source selection', design=<class 'panel.theme.materi..., name='Source', options=[np.str_('dyn'), ...], sizing_mode='stretch_width', value=np.str_('dyn'))
    [nav-139847733902224] RadioButtonGroup(description='Season selection', design=<class 'panel.theme.materi..., name='Season', options=[np.str_('DJF'), ...], sizing_mode='stretch_width', value=np.str_('DJF'))
    [nav-139847733899824] RadioButtonGroup(description='Model selection', design=<class 'panel.theme.materi..., name='Model', options=[np.str_('CESM2'), ...], sizing_mode='stretch_width', value=np.str_('CESM2'))
    [nav-139847745063920] IntSlider(design=<class 'panel.theme.materi..., end=150, name='Colormap max value', sizing_mode='stretch_width', start=1, value=50)